# PostgreSQL en Heroku

La página web es completamente funcional, podemos incluso añadir entradas al administrador:

![](docs/img13.png)

Pero hay un pequeño problema y es que [el sistema de ficheros de Heroku es efímero](https://devcenter.heroku.com/articles/dynos#ephemeral-filesystem), eso significa que todos los cambios se deshacen al reiniciar la aplicación y una base de datos SQLite nunca será persistente por el hecho de funcionar en un fichero.

Para ilustrar este inconveniente supongamos que realizamos una modificación del código, como un comentario abajo del `settings.py`:

```python
# Comentario en el settings que no afecta en nada
```

Añadimos el cambio al repositorio:

```bash
$ git add tutorial/settings.py
$ git commit -m "Actualizacion de prueba"
```

```
[main e4f1b1f] Actualizacion de prueba
 1 file changed, 2 insertions(+)
```

Si publicamos la nueva versión del proyecto con este insignificante cambio, éste se reiniciará y...

```bash
$ git push heroku main
```

![](docs/img14.png)

**¡BOOM!** Se habrá borrado la cuarta entrada y todos los cambios.

La solución es utilizar una base de datos en un servicio aislado y por suerte para nosotros como los de Heroku son buena gente nos regalan una para pruebas. Funciona sobre `PostgreSQL`, se llama `Hobby Dev` y tiene unos flamantes 10MB de tamaño. Ya sabéis, si queréis más hay que pagar. 

Esta base de datos se encuentra creada por defecto, podemos observar su configuración en el *dashboard*:

![](docs/img15.png)

Para hacer uso de ella en producción utilizaremos dos paquetes que nos facilitarán mucho la vida `python_decouple` y `dj_database_url`:

```bash
$ pipenv install python_decouple dj_database_url
``` 

Ahora podemos recuperar automáticamente la configuración de la base de datos utilizando como origen la cadena de configuración en las variables de entorno de la aplicación de Heroku:

![](docs/img16.png)

En la parte superior de `settings.py` importaremos ambos módulos:

```python
import dj_database_url
from decouple import config
```

Abajo del todo sobreescribiremos, solo para producción y de una forma poco elegante todo hay que decirlo, la configuración de la base de datos **PostgreSQL**:

```python
if not DEBUG:
    DATABASES = {
        'default': dj_database_url.config(
            default=config('DATABASE_URL')
        )
    }
```

Fijaros que concuerda la variable de entorno `DATABASE_URL` con la que se establece en la conifguración.

Publicamos los cambios:

```bash
$ git add . --all
$ git commit -m "Base de datos postgresql"
$ git push heroku main
```

Veremos que fallará al hacer el despliegue:

```
No module named 'psycopg2'
```

Para desplegar Django con PostgreSQL necesitamos dos cosas, primero instalar esta base de datos en nuestra máquina y luego el conector de Python llamado `psycopg2`, así que no hay más remedio que instalarla.

Si tenéis **Windows** tenéis que ir a la web https://postgresapp.com/downloads.html, descargar el instalador para vuestro sistema y seguir las instrucciones. 

En **Mac** puedo utilizar `brew`:

```bash
$ brew install postgresql
```

En **Linux** se puede utilizar `apt-get` o el gestor de turno:

```bash
$ sudo apt-get install postgresql
```

Una vez instalado, si el servicio está en marcha deberíamos tener acceso a él desde una terminal:

![](docs/img18.png)

En este punto podemos continuar con la configuración de PostgreSQL instalando `pyscopg2` en el entorno virtual:

```bash
$ pipenv install psycopg2-binary
``` 

Publicamos los cambios:

```bash
$ git add . --all
$ git commit -m "Paquete psycopg2"
$ git push heroku main
```

Si accedemos a la página web no funcionará, dará un error 500:

![](docs/img19.png)

Tranquilidad, el problema es que tenemos que realizar la migración inicial. 

Podemos ejecutar comandos de forma remota en el servicio de Heroku muy fácilmente:

```bash
$ heroku run python manage.py migrate
```

Listo, aunque sin usuario ni entradas... 

![](docs/img20.png)

Podemos crear un primer superusuario remotamente y empezar a añadir contenido persistente que no se borrará al actualizar el repositorio:

```bash
$ heroku run python manage.py createsuperuser
```

Esto es muy cómodo, podemos incluso utilizar la `shell` de forma remota (aunque con un poco de lag):

```bash
$ heroku run python manage.py shell
```

Si tenéis algun problema podéis revisar los últimos logs del servicio con el comando:

```bash
$ heroku logs -n 10
```

Para más información sobre la gestión de los servicios de Heroku os dejo la [documentación oficial](https://devcenter.heroku.com/articles/how-heroku-works), es todo un mundo.

En cualquier caso os felicito, ya tenemos el proyecto publicado en la nube, una tarea digna de un administrador profesional. 

Si os ha picado el gusanillo y queréis aprender más os animo a echar un vistazo a mi [curso práctico de Django](https://www.hektorprofe.net/cupon/django). A lo largo de tres proyectos voy enseñando más cosas sobre este genial framework, que aunque lo realicé hace un tiempo sigue siendo compatible con las nuevas versiones.

<br />
<img src="docs/fin.png" style="max-width:175px"/>
